In [127]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [128]:
from bs4 import BeautifulSoup as bs
import urllib.request
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import datetime
import os
import getpass

In [129]:
def get_URL(page) :
  url_before_page = 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EB%85%BC%EC%96%B4&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount='

  url_after_page= '&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%EB%85%BC%EC%96%B4&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%85%BC%EC%96%B4'

  URL = url_before_page + page + url_after_page
  return URL

In [130]:
def get_reference(URL): 
  # driver_path = os.path.join(ROOT_PATH, "chromedriver") 
  # driver = webdriver.Chrome(driver_path, options=webdriver.ChromeOptions().add_argument("headless")) 
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)

  driver.get(URL) 
  html = driver.page_source 
  soup = bs(html, "html.parser") 
  title = soup.find("h3", "title") 
  title_txt = title.get_text("", strip=True).split("=") 
  title_kor = re.sub("\n\b", "", str(title_txt[0]).strip()) 
  # title_eng = str(title_txt[1]).strip() 
  # txt_box = [] 
  # for text in soup.find_all("div", "text"): 
  #   txt = text.get_text("", strip=True) 
  #   txt_box.append(txt) 

  # txt_kor = txt_box[1] 
  # txt_eng = txt_box[3] 
  
  detail_box = [] 
  detail_info = soup.select( "#soptionview > div > div.thesisInfo > div.infoDetail.on > div.infoDetailL > ul > li > div > p" ) 
  for detail in detail_info: 
    detail_content = detail.get_text("", strip=True) 
    detail_wrap = [] 
    detail_wrap.append(detail_content) 
    detail_box.append(detail_wrap) 
  # author = ",".join(detail_box[0]) 
  # book = ( "".join(detail_box[2] + detail_box[3]).replace("\n", "").replace("\t", "").replace(" ", "") + " p." + "".join(detail_box[-2]) ) 
  keyword = ",".join(detail_box[6]) 
  # reference_data = pd.DataFrame( { "저자": [author], "국문 제목": [title_kor], "영문 제목": [title_eng], "수록지": [book], "핵심어": [keyword], "링크": [URL], } ) 
  reference_data = pd.DataFrame( {"국문 제목": [title_kor], "핵심어": [keyword]} ) 

  driver.close() 
  return reference_data



In [131]:
def save_csv(csv_path, data): 
  csv = csv_path.replace("/","\\") 
  if os.path.isfile(csv_path): 
    data.to_csv(csv, mode='a', header = False, index=False, encoding="utf-8-sig") 
  else: 
    data.to_csv(csv, mode='w',header = True, index=False, encoding="utf-8-sig")


In [132]:
def get_link (csv_name, page_num) :

  for i in range(page_num) :
    current_page = i*10
    URL = get_URL(str(current_page))
    source_code_from_URL = urllib.request.urlopen(URL)
    soup = bs(source_code_from_URL, 'lxml', from_encoding='utf-8')

    for j in range(10) :
      paper_link = soup.select('li > div.cont > p.title > a')[j]['href']
      paper_url = 'http://riss.or.kr' + paper_link
      
      reference_data = get_reference(paper_url)

      save_csv(csv_name, reference_data)

      

In [133]:
pages = 20

get_link('유학사상.csv', pages)
